In [ ]:
# Step 2: Import necessary components
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import random
from sentence_transformers import SentenceTransformer, util
import re

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Step 3: Improved DebateAI class with memory and better generation
class DebateAI:
    def __init__(self, model_name="gpt2-medium"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
        self.debate_history = []
        self.similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.current_stance = ""
        self.topic = ""

    def set_debate_topic(self, topic, stance):
        self.topic = topic
        self.current_stance = stance
        self.debate_history = [f"Debate Topic: {topic}\nMy Stance: {stance}"]

    def _generate_prompt(self, opponent_argument=None):
        prompt = "\n".join(self.debate_history[-3:])  # Keep last 3 exchanges
        if opponent_argument:
            prompt += f"\nOpponent: {opponent_argument}\nResponse:"
        else:
            prompt += "\nOpening Argument:"
        return prompt

    def generate_response(self, opponent_argument=None):
        prompt = self._generate_prompt(opponent_argument)

        inputs = self.tokenizer(prompt, return_tensors="pt").to(device)
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=400,
            num_return_sequences=1,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.2,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        new_response = response[len(prompt):].split("\n")[0].strip()
        new_response = self._postprocess_response(new_response)

        # Add to history
        if opponent_argument:
            self.debate_history.append(f"Opponent: {opponent_argument}")
            self.debate_history.append(f"Response: {new_response}")
        else:
            self.debate_history.append(f"Opening Argument: {new_response}")

        return new_response

    def _postprocess_response(self, text):
        # Remove incomplete sentences
        text = re.split(r'[.!?]', text)[0]
        # Remove markdown formatting
        text = re.sub(r'\*+', '', text)
        return text.strip()

    def validate_concession(self, user_input):
        user_input_lower = user_input.lower()
        concession_phrases = ["i concede", "i lose", "you win", "i give up"]
        return any(phrase in user_input_lower for phrase in concession_phrases)

In [ ]:
# Step 4: Enhanced debate flow with continuous interaction
def conduct_debate(topic):
    print(f"\n{' DEBATE START ':=^80}")
    print(f"Topic: {topic}\n")

    # Initialize debater
    debater = DebateAI(model_name="gpt2-medium")

    # Assign stances
    ai_stance = random.choice(["For", "Against"])
    user_stance = "Against" if ai_stance == "For" else "For"
    debater.set_debate_topic(topic, ai_stance)

    # AI opening statement
    ai_argument = debater.generate_response()
    print(f"[AI ({ai_stance})] {ai_argument}\n")

    # Debate loop
    turn = 0
    max_turns = 10  # Safety limit

    while turn < max_turns:
        user_input = input("[You] ")

        if debater.validate_concession(user_input):
            print("\n[AI] Thank you for the debate. Let's continue this discussion another time!")
            return

        # Generate AI response
        ai_response = debater.generate_response(user_input)
        print(f"\n[AI ({ai_stance})] {ai_response}\n")

        turn += 1

        # Check for AI concession (optional)
        if "concede" in ai_response.lower():
            print("\n[AI] It appears I cannot counter that argument. Well done!")
            return

    print("\n[Moderator] Maximum debate turns reached. Thank you both for your arguments!")

In [ ]:
# Step 5: Run the debate
topic = "Should porn be banned?"
conduct_debate(topic)


================================= DEBATE START =================================
Topic: Should porn be banned?



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[AI (For)] "I think pornography is a problem but I also believe it should not become illegal, that's my opinion


[AI (For)] Well we can't have the brainwashing without destroying brains and they don`t like being told what to do

